In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [0]:
X,Y,folds=DataPreparation('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6


In [0]:
train_idx = folds[0][0]
test_idx = folds[0][1]

print("Train Index shape: ", train_idx.shape)
print("Test Index shape: ", test_idx.shape)

X_train = X[train_idx]
Y_train = Y[train_idx]
X_test = X[test_idx]
Y_test = Y[test_idx]
print("Train X shape : ", X_train.shape)
print("Train Y shape : ", Y_train.shape)
print("Test shape : ", X_test.shape)
X_train = feature_extraction(X_train)
X_test = feature_extraction(X_test)
print("Train X shape : ", X_train.shape)
print("Train Y shape : ", Y_train.shape)
print("Test shape : ", X_test.shape)

Train Index shape:  (18757,)
Test Index shape:  (2089,)
Train X shape :  (18757, 100, 3)
Train Y shape :  (18757,)
Test shape :  (2089, 100, 3)
Train X shape :  (18757, 13)
Train Y shape :  (18757,)
Test shape :  (2089, 13)


In [0]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
def Model(num_classes):
  layers = [
      tf.keras.layers.Dense(128, activation = tf.nn.relu),
      tf.keras.layers.Dense(64, activation = tf.nn.relu),
      tf.keras.layers.Dense(32, activation = tf.nn.relu),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  train_idx = folds[0][0]
  test_idx = folds[0][1]
  X_train = X[train_idx]
  X_test = X[test_idx]
  Total_Ytrain = Y[train_idx]
  Total_Ytest = Y[test_idx]
  Total_Xtrain = feature_extraction(X_train)
  Total_Xtest = feature_extraction(X_test)

  for i in range(1, len(folds)):
          train_idx = folds[i][0]
          test_idx = folds[i][1]

          X_train = X[train_idx]
          X_test = X[test_idx]
          Y_train = Y[train_idx]
          Y_test = Y[test_idx]
          X_train = feature_extraction(X_train)
          X_test = feature_extraction(X_test)
          Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
          Total_Ytrain=np.append(Total_Ytrain, Y_train)
          Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
          Total_Ytest=np.append(Total_Ytest, Y_test)


  print(Total_Xtrain.shape, Total_Ytrain.shape)
  print(Total_Xtest.shape,Total_Ytest.shape)

  model = Model(num_classes)

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(Total_Xtrain, Total_Ytrain, batch_size=32, epochs=50)
  _,acc=model.evaluate(Total_Xtest,Total_Ytest)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/20
719/719 [==============================] - 1s 2ms/step - loss: 0.2140 - sparse_categorical_accuracy: 0.9466
Epoch 2/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0153 - sparse_categorical_accuracy: 0.9947
Epoch 3/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0131 - sparse_categorical_accuracy: 0.9956
Epoch 4/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0012 - sparse_categorical_accuracy: 0.9998
Epoch 5/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0129 - sparse_categorical_accuracy: 0.9965
Epoch 6/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0048 - sparse_categorical_accuracy: 0.9989
Epoch 7/20
719/719 [==============================] - 1s 2ms/step - loss: 2.4618e-04 - sparse_categorical_accuracy: 1.0000
Epoch 8/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0111 - 

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/20
719/719 [==============================] - 1s 2ms/step - loss: 0.1875 - sparse_categorical_accuracy: 0.9507
Epoch 2/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0177 - sparse_categorical_accuracy: 0.9937
Epoch 3/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9942
Epoch 4/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0061 - sparse_categorical_accuracy: 0.9979
Epoch 5/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0156 - sparse_categorical_accuracy: 0.9958
Epoch 6/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0099 - sparse_categorical_accuracy: 0.9970
Epoch 7/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0020 - sparse_categorical_accuracy: 0.9993
Epoch 8/20
719/719 [==============================] - 1s 2ms/step - loss: 1.0037e-04 - 

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(12015, 93) (12015,)
(1335, 93) (1335,)
Epoch 1/20
376/376 [==============================] - 1s 2ms/step - loss: 0.3759 - sparse_categorical_accuracy: 0.9091
Epoch 2/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0477 - sparse_categorical_accuracy: 0.9836
Epoch 3/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0211 - sparse_categorical_accuracy: 0.9922
Epoch 4/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0084 - sparse_categorical_accuracy: 0.9969
Epoch 5/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0186 - sparse_categorical_accuracy: 0.9951
Epoch 6/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9993
Epoch 7/20
376/376 [==============================] - 1s 2ms/step - loss: 0.0016 - sparse_categorical_accuracy: 0.9996
Epoch 8/20
376/376 [==============================] - 1s 2ms/step - loss: 3.6219e-04 - 

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/20
719/719 [==============================] - 1s 2ms/step - loss: 0.2313 - sparse_categorical_accuracy: 0.9485
Epoch 2/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0219 - sparse_categorical_accuracy: 0.9933
Epoch 3/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0118 - sparse_categorical_accuracy: 0.9962
Epoch 4/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0113 - sparse_categorical_accuracy: 0.9967
Epoch 5/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0079 - sparse_categorical_accuracy: 0.9974
Epoch 6/20
719/719 [==============================] - 1s 2ms/step - loss: 0.0072 - sparse_categorical_accuracy: 0.9978
Epoch 7/20
719/719 [==============================] - 1s 2ms/step - loss: 6.4342e-04 - sparse_categorical_accuracy: 0.9999
Epoch 8/20
719/719 [==============================] - 1s 2ms/step - loss: 9.8079e-0

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(88416, 25) (88416,)
(9824, 25) (9824,)
Epoch 1/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.4764 - sparse_categorical_accuracy: 0.8166
Epoch 2/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.2377 - sparse_categorical_accuracy: 0.8979
Epoch 3/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1892 - sparse_categorical_accuracy: 0.9149
Epoch 4/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1635 - sparse_categorical_accuracy: 0.9233
Epoch 5/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1468 - sparse_categorical_accuracy: 0.9302
Epoch 6/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1364 - sparse_categorical_accuracy: 0.9349
Epoch 7/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1256 - sparse_categorical_accuracy: 0.9391
Epoch 8/20
2763/2763 [==============================] - 5s 2ms/step - loss

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(127712, 25) (127712,)
(9824, 25) (9824,)
Epoch 1/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.4164 - sparse_categorical_accuracy: 0.8374
Epoch 2/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.2032 - sparse_categorical_accuracy: 0.9084
Epoch 3/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.1646 - sparse_categorical_accuracy: 0.9222
Epoch 4/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.1440 - sparse_categorical_accuracy: 0.9303
Epoch 5/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.1325 - sparse_categorical_accuracy: 0.9362
Epoch 6/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.1194 - sparse_categorical_accuracy: 0.9422
Epoch 7/20
3991/3991 [==============================] - 7s 2ms/step - loss: 0.1126 - sparse_categorical_accuracy: 0.9469
Epoch 8/20
3991/3991 [==============================] - 7s 2ms/step - lo

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(45342, 25) (45342,)
(5038, 25) (5038,)
Epoch 1/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.6414 - sparse_categorical_accuracy: 0.7707
Epoch 2/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.3147 - sparse_categorical_accuracy: 0.8665
Epoch 3/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.2468 - sparse_categorical_accuracy: 0.8937
Epoch 4/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.2119 - sparse_categorical_accuracy: 0.9070
Epoch 5/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.1812 - sparse_categorical_accuracy: 0.9179
Epoch 6/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.1690 - sparse_categorical_accuracy: 0.9220
Epoch 7/20
1417/1417 [==============================] - 3s 2ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9292
Epoch 8/20
1417/1417 [==============================] - 3s 2ms/step - loss

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(88416, 25) (88416,)
(9824, 25) (9824,)
Epoch 1/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.4671 - sparse_categorical_accuracy: 0.8206
Epoch 2/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.2397 - sparse_categorical_accuracy: 0.8959
Epoch 3/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1903 - sparse_categorical_accuracy: 0.9134
Epoch 4/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1669 - sparse_categorical_accuracy: 0.9207
Epoch 5/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1469 - sparse_categorical_accuracy: 0.9295
Epoch 6/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1370 - sparse_categorical_accuracy: 0.9346
Epoch 7/20
2763/2763 [==============================] - 5s 2ms/step - loss: 0.1277 - sparse_categorical_accuracy: 0.9379
Epoch 8/20
2763/2763 [==============================] - 5s 2ms/step - loss

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(34839, 13) (34839,)
(3871, 13) (3871,)
Epoch 1/50
1089/1089 [==============================] - 2s 2ms/step - loss: 1.1977 - sparse_categorical_accuracy: 0.5913
Epoch 2/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.9244 - sparse_categorical_accuracy: 0.6575
Epoch 3/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.8364 - sparse_categorical_accuracy: 0.6801
Epoch 4/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.7934 - sparse_categorical_accuracy: 0.6966
Epoch 5/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.7638 - sparse_categorical_accuracy: 0.7099
Epoch 6/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.7358 - sparse_categorical_accuracy: 0.7179
Epoch 7/50
1089/1089 [==============================] - 2s 2ms/step - loss: 0.7049 - sparse_categorical_accuracy: 0.7276
Epoch 8/50
1089/1089 [==============================] - 2s 2ms/step - loss:

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(58065, 13) (58065,)
(3871, 13) (3871,)
Epoch 1/50
1815/1815 [==============================] - 3s 2ms/step - loss: 1.1018 - sparse_categorical_accuracy: 0.6138
Epoch 2/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.8521 - sparse_categorical_accuracy: 0.6786
Epoch 3/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.7781 - sparse_categorical_accuracy: 0.7013
Epoch 4/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.7306 - sparse_categorical_accuracy: 0.7151
Epoch 5/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.6904 - sparse_categorical_accuracy: 0.7282
Epoch 6/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.6623 - sparse_categorical_accuracy: 0.7388
Epoch 7/50
1815/1815 [==============================] - 3s 2ms/step - loss: 0.6358 - sparse_categorical_accuracy: 0.7466
Epoch 8/50
1815/1815 [==============================] - 3s 2ms/step - loss:

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(34920, 13) (34920,)
(3880, 13) (3880,)
Epoch 1/50
1092/1092 [==============================] - 2s 2ms/step - loss: 1.2365 - sparse_categorical_accuracy: 0.5786
Epoch 2/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.9157 - sparse_categorical_accuracy: 0.6637
Epoch 3/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.8411 - sparse_categorical_accuracy: 0.6838
Epoch 4/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.7961 - sparse_categorical_accuracy: 0.6984
Epoch 5/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.7641 - sparse_categorical_accuracy: 0.7063
Epoch 6/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.7483 - sparse_categorical_accuracy: 0.7128
Epoch 7/50
1092/1092 [==============================] - 2s 2ms/step - loss: 0.7205 - sparse_categorical_accuracy: 0.7223
Epoch 8/50
1092/1092 [==============================] - 2s 2ms/step - loss:

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(19314, 13) (19314,)
(2146, 13) (2146,)
Epoch 1/50
604/604 [==============================] - 1s 2ms/step - loss: 1.6060 - sparse_categorical_accuracy: 0.4775
Epoch 2/50
604/604 [==============================] - 1s 2ms/step - loss: 1.0645 - sparse_categorical_accuracy: 0.6049
Epoch 3/50
604/604 [==============================] - 1s 2ms/step - loss: 0.9628 - sparse_categorical_accuracy: 0.6330
Epoch 4/50
604/604 [==============================] - 1s 2ms/step - loss: 0.9219 - sparse_categorical_accuracy: 0.6424
Epoch 5/50
604/604 [==============================] - 1s 2ms/step - loss: 0.8881 - sparse_categorical_accuracy: 0.6524
Epoch 6/50
604/604 [==============================] - 1s 2ms/step - loss: 0.8638 - sparse_categorical_accuracy: 0.6617
Epoch 7/50
604/604 [==============================] - 1s 2ms/step - loss: 0.8394 - sparse_categorical_accuracy: 0.6697
Epoch 8/50
604/604 [==============================] - 1s 2ms/step - loss: 0.8053 - sparse

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(187614, 13) (187614,)
(20846, 13) (20846,)
Epoch 1/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.3305 - sparse_categorical_accuracy: 0.8802
Epoch 2/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9318
Epoch 3/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1459 - sparse_categorical_accuracy: 0.9482
Epoch 4/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1243 - sparse_categorical_accuracy: 0.9564
Epoch 5/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1096 - sparse_categorical_accuracy: 0.9614
Epoch 6/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.0968 - sparse_categorical_accuracy: 0.9655
Epoch 7/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.0877 - sparse_categorical_accuracy: 0.9689
Epoch 8/50
5863/5863 [==============================] - 12s 2ms/s

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(729610, 13) (729610,)
(20846, 13) (20846,)
Epoch 1/50
22801/22801 [==============================] - 42s 2ms/step - loss: 0.1952 - sparse_categorical_accuracy: 0.9302
Epoch 2/50
22801/22801 [==============================] - 43s 2ms/step - loss: 0.0916 - sparse_categorical_accuracy: 0.9678
Epoch 3/50
22801/22801 [==============================] - 42s 2ms/step - loss: 0.0689 - sparse_categorical_accuracy: 0.9758
Epoch 4/50
22801/22801 [==============================] - 43s 2ms/step - loss: 0.0555 - sparse_categorical_accuracy: 0.9803
Epoch 5/50
22801/22801 [==============================] - 45s 2ms/step - loss: 0.0473 - sparse_categorical_accuracy: 0.9835
Epoch 6/50
22801/22801 [==============================] - 44s 2ms/step - loss: 0.0422 - sparse_categorical_accuracy: 0.9851
Epoch 7/50
22801/22801 [==============================] - 43s 2ms/step - loss: 0.0376 - sparse_categorical_accuracy: 0.9869
Epoch 8/50
22801/22801 [===========================

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(187614, 13) (187614,)
(20846, 13) (20846,)
Epoch 1/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.3256 - sparse_categorical_accuracy: 0.8831
Epoch 2/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1771 - sparse_categorical_accuracy: 0.9360
Epoch 3/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1328 - sparse_categorical_accuracy: 0.9528
Epoch 4/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.1128 - sparse_categorical_accuracy: 0.9598
Epoch 5/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.0974 - sparse_categorical_accuracy: 0.9653
Epoch 6/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.0884 - sparse_categorical_accuracy: 0.9683
Epoch 7/50
5863/5863 [==============================] - 11s 2ms/step - loss: 0.0805 - sparse_categorical_accuracy: 0.9712
Epoch 8/50
5863/5863 [==============================] - 11s 2ms/s

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(94644, 13) (94644,)
(10516, 13) (10516,)
Epoch 1/50
2958/2958 [==============================] - 5s 2ms/step - loss: 0.3990 - sparse_categorical_accuracy: 0.8572
Epoch 2/50
2958/2958 [==============================] - 5s 2ms/step - loss: 0.2662 - sparse_categorical_accuracy: 0.9037
Epoch 3/50
2958/2958 [==============================] - 5s 2ms/step - loss: 0.2053 - sparse_categorical_accuracy: 0.9246
Epoch 4/50
2958/2958 [==============================] - 5s 2ms/step - loss: 0.1742 - sparse_categorical_accuracy: 0.9357
Epoch 5/50
2958/2958 [==============================] - 6s 2ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9462
Epoch 6/50
2958/2958 [==============================] - 6s 2ms/step - loss: 0.1354 - sparse_categorical_accuracy: 0.9514
Epoch 7/50
2958/2958 [==============================] - 6s 2ms/step - loss: 0.1216 - sparse_categorical_accuracy: 0.9568
Epoch 8/50
2958/2958 [==============================] - 5s 2ms/step - loss

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(10233, 25) (10233,)
(1137, 25) (1137,)
Epoch 1/50
320/320 [==============================] - 1s 2ms/step - loss: 1.0259 - sparse_categorical_accuracy: 0.6037
Epoch 2/50
320/320 [==============================] - 1s 2ms/step - loss: 0.6086 - sparse_categorical_accuracy: 0.7525
Epoch 3/50
320/320 [==============================] - 1s 2ms/step - loss: 0.4600 - sparse_categorical_accuracy: 0.8099
Epoch 4/50
320/320 [==============================] - 1s 2ms/step - loss: 0.3893 - sparse_categorical_accuracy: 0.8388
Epoch 5/50
320/320 [==============================] - 1s 2ms/step - loss: 0.3523 - sparse_categorical_accuracy: 0.8535
Epoch 6/50
320/320 [==============================] - 1s 2ms/step - loss: 0.3017 - sparse_categorical_accuracy: 0.8788
Epoch 7/50
320/320 [==============================] - 1s 2ms/step - loss: 0.2714 - sparse_categorical_accuracy: 0.8911
Epoch 8/50
320/320 [==============================] - 1s 2ms/step - loss: 0.2325 - 

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(7959, 25) (7959,)
(1137, 25) (1137,)
Epoch 1/50
249/249 [==============================] - 0s 2ms/step - loss: 1.1006 - sparse_categorical_accuracy: 0.5875
Epoch 2/50
249/249 [==============================] - 0s 2ms/step - loss: 0.6347 - sparse_categorical_accuracy: 0.7333
Epoch 3/50
249/249 [==============================] - 0s 2ms/step - loss: 0.5021 - sparse_categorical_accuracy: 0.7981
Epoch 4/50
249/249 [==============================] - 0s 2ms/step - loss: 0.4369 - sparse_categorical_accuracy: 0.8217
Epoch 5/50
249/249 [==============================] - 0s 2ms/step - loss: 0.3791 - sparse_categorical_accuracy: 0.8509
Epoch 6/50
249/249 [==============================] - 0s 2ms/step - loss: 0.3314 - sparse_categorical_accuracy: 0.8671
Epoch 7/50
249/249 [==============================] - 0s 2ms/step - loss: 0.3138 - sparse_categorical_accuracy: 0.8731
Epoch 8/50
249/249 [==============================] - 0s 2ms/step - loss: 0.2767 - sp

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(5544, 25) (5544,)
(616, 25) (616,)
Epoch 1/50
174/174 [==============================] - 0s 2ms/step - loss: 0.9807 - sparse_categorical_accuracy: 0.5915
Epoch 2/50
174/174 [==============================] - 0s 2ms/step - loss: 0.6208 - sparse_categorical_accuracy: 0.7459
Epoch 3/50
174/174 [==============================] - 0s 2ms/step - loss: 0.4806 - sparse_categorical_accuracy: 0.8146
Epoch 4/50
174/174 [==============================] - 0s 2ms/step - loss: 0.3644 - sparse_categorical_accuracy: 0.8631
Epoch 5/50
174/174 [==============================] - 0s 2ms/step - loss: 0.3027 - sparse_categorical_accuracy: 0.8894
Epoch 6/50
174/174 [==============================] - 0s 2ms/step - loss: 0.2583 - sparse_categorical_accuracy: 0.9116
Epoch 7/50
174/174 [==============================] - 0s 2ms/step - loss: 0.2281 - sparse_categorical_accuracy: 0.9174
Epoch 8/50
174/174 [==============================] - 0s 2ms/step - loss: 0.1982 - spar

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(10233, 25) (10233,)
(1137, 25) (1137,)
Epoch 1/50
320/320 [==============================] - 1s 2ms/step - loss: 0.9587 - sparse_categorical_accuracy: 0.6367
Epoch 2/50
320/320 [==============================] - 1s 2ms/step - loss: 0.5645 - sparse_categorical_accuracy: 0.7738
Epoch 3/50
320/320 [==============================] - 1s 2ms/step - loss: 0.4449 - sparse_categorical_accuracy: 0.8198
Epoch 4/50
320/320 [==============================] - 1s 2ms/step - loss: 0.3859 - sparse_categorical_accuracy: 0.8423
Epoch 5/50
320/320 [==============================] - 1s 2ms/step - loss: 0.3294 - sparse_categorical_accuracy: 0.8671
Epoch 6/50
320/320 [==============================] - 1s 2ms/step - loss: 0.2997 - sparse_categorical_accuracy: 0.8811
Epoch 7/50
320/320 [==============================] - 1s 2ms/step - loss: 0.2622 - sparse_categorical_accuracy: 0.8990
Epoch 8/50
320/320 [==============================] - 1s 2ms/step - loss: 0.2302 - 

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(33939, 25) (33939,)
(3771, 25) (3771,)
Epoch 1/50
1061/1061 [==============================] - 2s 2ms/step - loss: 2.7014 - sparse_categorical_accuracy: 0.2083
Epoch 2/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.9591 - sparse_categorical_accuracy: 0.3774
Epoch 3/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.6280 - sparse_categorical_accuracy: 0.4671
Epoch 4/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.3878 - sparse_categorical_accuracy: 0.5404
Epoch 5/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.2170 - sparse_categorical_accuracy: 0.5946
Epoch 6/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.0831 - sparse_categorical_accuracy: 0.6376
Epoch 7/50
1061/1061 [==============================] - 2s 2ms/step - loss: 0.9801 - sparse_categorical_accuracy: 0.6697
Epoch 8/50
1061/1061 [==============================] - 2s 2ms/step 

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(26397, 25) (26397,)
(3771, 25) (3771,)
Epoch 1/50
825/825 [==============================] - 2s 2ms/step - loss: 2.8798 - sparse_categorical_accuracy: 0.2109
Epoch 2/50
825/825 [==============================] - 2s 2ms/step - loss: 2.0468 - sparse_categorical_accuracy: 0.3431
Epoch 3/50
825/825 [==============================] - 2s 2ms/step - loss: 1.7181 - sparse_categorical_accuracy: 0.4340
Epoch 4/50
825/825 [==============================] - 2s 2ms/step - loss: 1.5076 - sparse_categorical_accuracy: 0.4978
Epoch 5/50
825/825 [==============================] - 2s 2ms/step - loss: 1.3494 - sparse_categorical_accuracy: 0.5514
Epoch 6/50
825/825 [==============================] - 2s 2ms/step - loss: 1.2195 - sparse_categorical_accuracy: 0.5900
Epoch 7/50
825/825 [==============================] - 2s 2ms/step - loss: 1.1160 - sparse_categorical_accuracy: 0.6215
Epoch 8/50
825/825 [==============================] - 2s 2ms/step - loss: 1.0486 -

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(33939, 25) (33939,)
(3771, 25) (3771,)
Epoch 1/50
1061/1061 [==============================] - 2s 2ms/step - loss: 2.9829 - sparse_categorical_accuracy: 0.1911
Epoch 2/50
1061/1061 [==============================] - 2s 2ms/step - loss: 2.1284 - sparse_categorical_accuracy: 0.3244
Epoch 3/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.7455 - sparse_categorical_accuracy: 0.4204
Epoch 4/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.5115 - sparse_categorical_accuracy: 0.4901
Epoch 5/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.3255 - sparse_categorical_accuracy: 0.5519
Epoch 6/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.1817 - sparse_categorical_accuracy: 0.6020
Epoch 7/50
1061/1061 [==============================] - 2s 2ms/step - loss: 1.0809 - sparse_categorical_accuracy: 0.6355
Epoch 8/50
1061/1061 [==============================] - 2s 2ms/step 

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(18432, 25) (18432,)
(2048, 25) (2048,)
Epoch 1/50
576/576 [==============================] - 1s 2ms/step - loss: 3.1151 - sparse_categorical_accuracy: 0.1665
Epoch 2/50
576/576 [==============================] - 1s 2ms/step - loss: 2.2880 - sparse_categorical_accuracy: 0.2826
Epoch 3/50
576/576 [==============================] - 1s 2ms/step - loss: 1.9812 - sparse_categorical_accuracy: 0.3709
Epoch 4/50
576/576 [==============================] - 1s 2ms/step - loss: 1.7370 - sparse_categorical_accuracy: 0.4441
Epoch 5/50
576/576 [==============================] - 1s 2ms/step - loss: 1.5326 - sparse_categorical_accuracy: 0.4976
Epoch 6/50
576/576 [==============================] - 1s 2ms/step - loss: 1.3755 - sparse_categorical_accuracy: 0.5450
Epoch 7/50
576/576 [==============================] - 1s 2ms/step - loss: 1.2420 - sparse_categorical_accuracy: 0.5872
Epoch 8/50
576/576 [==============================] - 1s 2ms/step - loss: 1.1428 -